In [ ]:
import numpy as np
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/RL/new/states.data', 'rb') as filehandle:
    # read the data as binary data stream
    STATES = pickle.load(filehandle)

with open('/content/drive/MyDrive/RL/new/transition_prob.data', 'rb') as filehandle:
    # read the data as binary data stream
    transition_probability = pickle.load(filehandle)

In [ ]:
STATES = np.asarray(STATES) 

In [ ]:
len(STATES)

3381

In [ ]:
def check_game_over_3(markers):
  game_over = False
  tie = False
  winner =0
  x_pos = 0
  for x in markers:
    #check columns
    if sum(x) == 3:
      winner = 1
      game_over = True
    if sum(x) == -3:
      winner = 2
      game_over = True
    
    #check rows
    if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == 3:
      winner = 1
      game_over = True
    if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == -3:
      winner = 2
      game_over = True
    x_pos += 1

  #check cross
  if markers[0][0] + markers[1][1] + markers [2][2] == 3 or markers[2][0] + markers[1][1] + markers [0][2] == 3:
    winner = 1
    game_over = True
  if markers[0][0] + markers[1][1] + markers [2][2] == -3 or markers[2][0] + markers[1][1] + markers [0][2] == -3:
    winner = 2
    game_over = True

  #check for tie
  if game_over == False:
    tie = True
    for row in markers:
      for i in row:
        if i == 0:
          tie = False
    #if it is a tie, then call game over and set winner to 0 (no one)
    if tie == True:
      game_over = True
      winner = 0
  return game_over,winner,tie
  # if game_over == False:

In [ ]:
def reward(y):
    r=0
    winner = 0
    y = np.array(y).reshape(3,3)
    y[np.where(y==2)] = -1
    go,winner,tie = check_game_over_3(y)
    if go == True:
        if winner == 1:
            r = 100
        elif winner == 2:
            r = -100
    if tie == True:
        r = 10
    return r

In [ ]:
REWARDS = np.array([0]*len(STATES))
for i in range(len(STATES)):
  REWARDS[i] = reward(STATES[i])

# Generating arbitrary policy

In [ ]:
pi_star = np.array([0]*len(STATES))
W = np.array([0.0]*len(STATES))
gamma = 0.1

In [ ]:
for i,s in enumerate(STATES):
  if 0 in s:
    pi_star[i] = np.where(s==0)[0][0]

In [ ]:
pi0 = pi_star.copy()

In [ ]:
for k in range(10):
  for i,s in enumerate(STATES):
    action = pi_star[i]
    rt = np.dot(transition_probability[i][action],REWARDS)
    W[i] = rt + gamma*np.dot(transition_probability[i][action],W)
    Re = [-1000]*9
    for a in range(9):
      if s[a] == 0:
        Re[a] = np.dot(transition_probability[i][a],REWARDS+gamma*W)
    pi_star[i] = np.argmax(Re)
  print(pi_star)

[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]
[0 6 3 ... 0 0 0]


In [ ]:
pi_star[1250]

2

In [ ]:
with open('p_policy_iteration.data','wb') as filehandle:
  pickle.dump(pi_star, filehandle)